Deflacionar valores nomianais de importações e exportações. Valores estão em dolares.

In [1]:
import pandas as pd
from pathlib import Path
import plotly.express as px


data_path = Path().resolve().parent / "data"
imp = pd.read_csv(data_path / "importacoes.csv", sep=";", encoding="latin1")
exp = pd.read_csv(data_path / "exportacoes.csv", sep=";", encoding="latin1")

imp_oz = imp.loc[imp['no_mun_min'].str.contains("Osasco")].copy()
exp_oz = exp.loc[exp['no_mun_min'].str.contains("Osasco")].copy()

imp_fob_anual_oz = imp_oz.loc[imp_oz["co_ano"] != 2025].groupby("co_ano", as_index=False).agg({"vl_fob":"sum"})
exp_fob_anual_oz = exp_oz.loc[exp_oz["co_ano"] != 2025].groupby("co_ano", as_index=False).agg({"vl_fob":"sum"})

In [2]:
px.line(
    imp_fob_anual_oz,
    x="co_ano",
    y="vl_fob",
    markers=True
)

In [3]:
px.line(
    exp_fob_anual_oz,
    x="co_ano",
    y="vl_fob",
    markers=True
)

In [4]:
imp_oz.query("co_ano == 2024").groupby("no_sh4_por", as_index=False).agg(
    {"vl_fob": "sum"}
).sort_values("vl_fob", ascending=False).head(10)

,no_sh4_por,vl_fob
234,"Instrumentos e aparelhos para medicina, cirurg...",118701248
380,Partes e acessórios dos veículos automóveis da...,104032124
49,"Artigos e aparelhos ortopédicos, incluídas as ...",78491324
391,"Peixes frescos ou refrigerados, exceto os filé...",28502942
191,Filés de peixes e outra carne de peixes (mesmo...,18051269
546,"Óleos de petróleo ou de minerais betuminosos, ...",16533984
443,Queijos e requeijão,16044089
63,"Bombas de ar ou de vácuo, compressores de ar o...",14143522
56,"Azeite de oliveira e respectivas fracções, mes...",11678516
64,"Bombas para líquidos, mesmo com dispositivo me...",10870767


In [5]:
exp_oz.query("co_ano == 2024").groupby("no_sh4_por", as_index=False).agg(
    {"vl_fob": "sum"}
).sort_values("vl_fob", ascending=False).head(10)

,no_sh4_por,vl_fob
201,Partes e acessórios dos veículos automóveis da...,21206027
113,"Instrumentos e aparelhos para medicina, cirurg...",11840816
5,Agentes orgânicos de superfície (exceto sabões...,6107625
70,"Cordas, cabos, entrançados, lingas e artefacto...",4864876
211,Polímeros de cloreto de vinilo ou de outras ol...,4714240
279,"Óleos de petróleo ou de minerais betuminosos, ...",3988187
232,"Refrigeradores, congeladores (freezers) e outr...",3886686
199,"Partes dos aparelhos das posições 88.01, 88.02...",3868910
150,"Mástique de vidraceiro, cimentos de resina e o...",3653699
148,"Máquinas e aparelhos, para trabalhar borracha ...",3619339


In [6]:
import pandas as pd
import requests


def get_cambio(start_year=2000):
    url = f"https://api.bcb.gov.br/dados/serie/bcdata.sgs.1/dados?formato=json"
    response = requests.get(url).json()

    df = pd.DataFrame(response)
    df["data"] = pd.to_datetime(df["data"], dayfirst=True)
    df["ano"] = df["data"].dt.year
    df["cambio_nominal"] = df["valor"].astype(float)

    # Média do câmbio por ano
    df = (
        df[df["ano"] >= start_year]
        .groupby("ano")["cambio_nominal"]
        .mean()
        .reset_index()
    )
    return df


def get_ipca(start_year=2000):
    url = f"https://api.bcb.gov.br/dados/serie/bcdata.sgs.433/dados?formato=json"
    response = requests.get(url).json()

    df = pd.DataFrame(response)
    df["data"] = pd.to_datetime(df["data"], dayfirst=True)
    df["ano"] = df["data"].dt.year
    df["ipca"] = df["valor"].astype(float)

    # Agrupar por ano e pegar o último valor do ano
    df = df[df["ano"] >= start_year].groupby("ano")["ipca"].last().reset_index()
    return df


df_cambio = get_cambio()
df_ipca = get_ipca()

In [7]:
# # Juntar os dados de IPCA, câmbio e valores do comércio exterior
# df = df_comex.merge(df_ipca, on="ano", how="left").merge(df_cambio, on="ano", how="left")

# # Definir o IPCA de referência (exemplo: 2023)
# ipca_referencia = df[df["ano"] == 2023]["ipca"].values[0]

# # Calcular a taxa de câmbio real
# df["cambio_real"] = (df["cambio_nominal"] * ipca_referencia) / df["ipca"]

# # Ajustar os valores para moeda constante
# df["valor_real"] = (df["valor_nominal"] * df["cambio_nominal"]) / df["cambio_real"]

# # Exibir os dados processados
# import ace_tools as tools
# tools.display_dataframe_to_user(name="Dados Deflacionados", dataframe=df)
